# Lightkurve periodogram & BLS-only notebook (Coarse → Refined)

This notebook:
- loads your CSV and forces use of the `Flux` column (second column) as requested,
- runs a two-step Box‑Least‑Squares (BLS) search (coarse -> refined),
- overlays Lightkurve's periodogram model if available, and
- produces a time-domain zoom and phase-folded, binned transit plot for the best candidate.

Defaults: period range 0.5–25 days; durations grid 0.005–0.5 days.

Generated: 2025-09-18 07:42 UTC

---



In [ ]:
# Install required packages if needed (uncomment to run)
# !pip install lightkurve astropy numpy pandas matplotlib scipy
# Restart kernel after major installs if prompted.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.timeseries import BoxLeastSquares
try:
    import lightkurve as lk
except Exception as e:
    print('lightkurve not available:', e)

%matplotlib inline
plt.rcParams['figure.figsize'] = (10,4)


In [ ]:
# === EDIT THIS PATH IF YOUR FILE IS LOCATED ELSEWHERE ===
DATA_PATH = "tic437011608flattened-2min.csv"

df = pd.read_csv(DATA_PATH)
print('Columns found:', df.columns.tolist())
display(df.head())

# Detect time column (auto-detect based on 'time' or 'bjd')
time_col_candidates = [c for c in df.columns if 'time' in c.lower() or 'bjd' in c.lower()]
time_col = time_col_candidates[0] if time_col_candidates else df.columns[0]

# Force use of the 'Flux' column (2nd column) instead of Flattened Flux
if 'Flux' in df.columns:
    flux_col = 'Flux'
else:
    flux_col = df.columns[1]  # fallback to second column

time_rel = df[time_col].values.astype(float)
time_bjd = time_rel + 2457000.0
flux = df[flux_col].values.astype(float)

print(f"Using time column: '{time_col}' (interpreted as BJD-2457000).")
print(f"Using flux column: '{flux_col}'. Number of points: {len(time_rel)}")


In [ ]:
# Create a LightKurve LightCurve object (optional)
try:
    lc = lk.LightCurve(time=time_rel, flux=flux)
    print('LightKurve object created.')
except Exception as e:
    print('lightkurve not available or failed to create LightCurve object:', e)
    lc = None


In [ ]:
# ===== Two-step BLS: coarse -> refined =====
# PARAMETERS (tweakable)
min_period = 0.5      # days
max_period = 25.0     # days

# Coarse grid (fast)
n_coarse = 3000       # fewer trial periods to run quickly
durations = np.linspace(0.005, 0.5, 30)   # grid of durations (days) for autopower scan

# Refined grid (local around top peaks)
n_refined = 5000      # resolution for refined search (only around candidate peaks)
refine_width = 0.02   # fractional window around coarse peak to refine (e.g. 0.02 = +-2%)

print('Data points:', len(time_rel))
print('Coarse periods:', n_coarse, 'Refined resolution:', n_refined)

# --- Coarse BLS using astropy BoxLeastSquares.autopower ---
bls = BoxLeastSquares(time_rel, flux)
print('Running coarse autopower... (this is reasonably fast)')
try:
    coarse_result = bls.autopower(durations, minimum_period=min_period, maximum_period=max_period)
    period_c = coarse_result.period
    power_c = coarse_result.power
    plt.figure(); plt.plot(period_c, power_c, lw=0.8)
    plt.xlabel('Period (days)'); plt.ylabel('BLS power')
    plt.title('Coarse BLS periodogram'); plt.xlim(min_period, max_period)
    plt.show()
    # pick top candidate periods (local maxima)
    from scipy.signal import find_peaks
    peaks, _ = find_peaks(power_c, height=np.nanmedian(power_c) + 3*np.nanstd(power_c))
    if len(peaks) == 0:
        best_idx_coarse = np.nanargmax(power_c)
        candidate_periods = [period_c[best_idx_coarse]]
    else:
        peak_powers = power_c[peaks]
        order = np.argsort(peak_powers)[::-1]
        candidate_periods = list(period_c[peaks][order][:3])
    print('Candidate periods from coarse search (days):', candidate_periods)
except Exception as e:
    print('Coarse BLS autopower failed:', e)
    period_c = np.linspace(min_period, max_period, n_coarse)
    power_c = np.zeros_like(period_c)
    candidate_periods = []

# --- Refined searches around each candidate ---
refined_results = []
for p0 in candidate_periods:
    pmin = p0*(1.0 - refine_width)
    pmax = p0*(1.0 + refine_width)
    print(f'Refining around {p0:.6f} d -> window [{pmin:.6f}, {pmax:.6f}]')
    try:
        refined_autopower = bls.autopower(durations, minimum_period=pmin, maximum_period=pmax)
        per_r = refined_autopower.period
        pow_r = refined_autopower.power
        best_idx = np.nanargmax(pow_r)
        best_p = per_r[best_idx]
        best_pow = pow_r[best_idx]
        best_dur = refined_autopower.duration[best_idx] if hasattr(refined_autopower, 'duration') else np.nan
        refined_results.append({'start':pmin, 'stop':pmax, 'best_period':float(best_p),
                                'best_power':float(best_pow), 'best_duration':float(best_dur)})
        plt.figure(); plt.plot(per_r, pow_r, lw=0.8)
        plt.xlabel('Period (days)'); plt.ylabel('BLS power')
        plt.title(f'Refined BLS around {p0:.6f} d (best {best_p:.6f} d)')
        plt.show()
    except Exception as e:
        print('Refined autopower failed for window', pmin, pmax, e)

print('\nRefined results (summary):')
for r in refined_results:
    print(r)

# If lightkurve is available, also attempt a lightkurve BLS as a convenience check
if lc is not None:
    try:
        print('\nAttempting Lightkurve BLS convenience call (may differ slightly):')
        lk_pg = lc.to_periodogram(method='bls', minimum_period=min_period, maximum_period=max_period)
        display(lk_pg)
        print('Lightkurve BLS best period:', lk_pg.period_at_max_power)
    except Exception as e:
        print('Lightkurve BLS convenience call failed:', e)


In [ ]:
# --- BLS diagnostic: simple autopower call if you want a separate BLS diagnostic cell ---
# (This cell intentionally repeats minimal BLS autopower call and prints best period/duration)
try:
    durations = np.linspace(0.005, 0.5, 50)
    bls_result = bls.autopower(durations, minimum_period=min_period, maximum_period=max_period)
    plt.figure(); plt.plot(bls_result.period, bls_result.power); plt.xlabel('Period (days)'); plt.ylabel('BLS power'); plt.title('BLS periodogram (autopower)'); plt.show()
    best_idx = np.nanargmax(bls_result.power)
    print('BLS best period:', bls_result.period[best_idx], 'duration:', bls_result.duration[best_idx] if hasattr(bls_result, 'duration') else None)
except Exception as e:
    print('BLS diagnostic failed:', e)


In [ ]:
# Zoom + phase-folded binned transit plot with BLS model overlay (uses Lightkurve pg.model if available)
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Determine best period P and duration dur from refined_results or bls_result
P = None
dur = None
if 'refined_results' in globals() and len(refined_results) > 0:
    P = refined_results[0]['best_period']
    dur = refined_results[0].get('best_duration', None)
elif 'bls_result' in globals():
    try:
        idx = np.nanargmax(bls_result.power)
        P = float(bls_result.period[idx])
        dur = float(bls_result.duration[idx]) if hasattr(bls_result, 'duration') else None
    except Exception:
        P = None

if P is None:
    # fallback to lightkurve periodogram if created
    if 'lk_pg' in globals() and lk_pg is not None:
        try:
            P = float(lk_pg.period_at_max_power)
        except Exception:
            P = None

if P is None:
    raise RuntimeError('No best BLS period found yet. Run the BLS cells first.')

if dur is None or not np.isfinite(dur):
    dur = max(0.05 * P, 0.01)

# Try to get a transit epoch t0 from bls_result or lk_pg if possible, else estimate from folded minimum
t0 = None
if 'bls_result' in globals() and hasattr(bls_result, 'transit_time'):
    try:
        t0 = float(bls_result.transit_time[np.nanargmax(bls_result.power)])
    except Exception:
        t0 = None
if t0 is None and 'lk_pg' in globals() and hasattr(lk_pg, 'transit_time'):
    try:
        t0 = float(lk_pg.transit_time)
    except Exception:
        t0 = None

if t0 is None:
    tref = time_rel.min()
    phases = ((time_rel - tref) % P) / P
    med = stats.binned_statistic(phases, flux, statistic='median', bins=200).statistic
    edges = stats.binned_statistic(phases, flux, statistic='median', bins=200).bin_edges
    min_idx = np.nanargmin(med)
    phase_min = 0.5*(edges[min_idx] + edges[min_idx+1])
    t0 = tref + phase_min * P

# pick central transit
k = np.round((np.median(time_rel) - t0)/P).astype(int)
t_center = t0 + k * P

# zoom window
window_days = max(5 * dur, 0.05)
tmin_zoom = t_center - window_days
tmax_zoom = t_center + window_days
mask_zoom = (time_rel >= tmin_zoom) & (time_rel <= tmax_zoom)

# Prepare model using Lightkurve pg.model if present, else use BoxLeastSquares.model
model_t = None; model_f = None
if 'lk_pg' in globals() and lk_pg is not None:
    try:
        lc_model = lk_pg.model(time=lc.time, period=lk_pg.period_at_max_power)
        model_t = np.asarray(lc_model.time.value)
        model_f = np.asarray(lc_model.flux.value)
    except Exception:
        pass

if model_t is None:
    try:
        bls_inst = BoxLeastSquares(time_rel, flux)
        model_t = np.linspace(tmin_zoom, tmax_zoom, 2000)
        model_f = bls_inst.model(model_t, P, dur, t_center)
    except Exception:
        model_t = np.linspace(tmin_zoom, tmax_zoom, 2000)
        model_f = np.ones_like(model_t) * np.nan

# Plot time zoom + folded binned with model overlay
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(12,8))
ax1.plot(time_rel[mask_zoom], flux[mask_zoom], '.', ms=3, alpha=0.6, color='k', label='data (zoom)')
ax1.plot(model_t, model_f, '-', color='C1', lw=2, label='BLS model')
ax1.axvline(t_center, color='0.5', ls=':', label='transit center')
ax1.set_xlim(tmin_zoom, tmax_zoom)
ax1.set_ylabel('Normalized Flux')
ax1.set_title(f'Time-domain zoom: Transit near t = {t_center:.5f} (P = {P:.6f} d, dur={dur:.4f} d)')
ax1.legend()

# Fold and bin around transit
phase_all = ((time_rel - t_center) / P)
phase_all = (phase_all + 0.5) % 1.0 - 0.5
phase_vals = phase_all[mask_zoom]
flux_vals = flux[mask_zoom]

nbins = 60
bin_res = stats.binned_statistic(phase_vals, flux_vals, statistic='median', bins=nbins)
bin_centers = 0.5*(bin_res.bin_edges[:-1] + bin_res.bin_edges[1:])
bin_meds = bin_res.statistic
bin_std = stats.binned_statistic(phase_vals, flux_vals, statistic='std', bins=nbins).statistic
bin_count = stats.binned_statistic(phase_vals, flux_vals, statistic='count', bins=nbins).statistic
bin_err = np.where(bin_count>0, bin_std/np.sqrt(bin_count), np.nan)

# fold model to phase
model_phase = ((model_t - t_center)/P)
model_phase = (model_phase + 0.5) % 1.0 - 0.5
sort_idx = np.argsort(model_phase)
model_phase_plot = np.concatenate([model_phase[sort_idx]-1.0, model_phase[sort_idx], model_phase[sort_idx]+1.0])
model_flux_plot = np.concatenate([model_f[sort_idx], model_f[sort_idx], model_f[sort_idx]])

# duplicate bins for two cycles
bin_centers_dup = np.concatenate([bin_centers-1.0, bin_centers, bin_centers+1.0])
bin_meds_dup = np.concatenate([bin_meds, bin_meds, bin_meds])
bin_err_dup = np.concatenate([bin_err, bin_err, bin_err])

ax2.plot(np.concatenate([phase_vals-1.0, phase_vals, phase_vals+1.0]),
         np.concatenate([flux_vals, flux_vals, flux_vals]), '.', ms=2, color='0.2', alpha=0.25)
ax2.errorbar(bin_centers_dup, bin_meds_dup, yerr=bin_err_dup, fmt='o', ms=5, mec='k', mfc='C0', ecolor='C0', capsize=2, label='binned median')
ax2.plot(model_phase_plot, model_flux_plot, '-', color='C1', lw=2, label='folded BLS model')
ax2.set_xlim(-0.2, 0.2)
ax2.set_xlabel('Phase (centered on transit) [cycles]')
ax2.set_ylabel('Normalized Flux')
ax2.legend()
plt.tight_layout()
plt.show()


---
Notes:
- This notebook forces use of the 'Flux' column as requested, and focuses on Box-Least-Squares (BLS) methods only.
- Adjust n_coarse, n_refined, durations grid, and refine_width to trade speed vs resolution.
- If you want a Lomb-Scargle comparison cell added back in, let me know and I'll re-add it.
---
